In [1]:
import os
import sys
import inspect
from dotenv import load_dotenv

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
os.environ['SERPAPI_API_KEY'] = ''

In [4]:
# to import backend module
currentdir = os.path.dirname(os.path.abspath(globals()['_dh'][0]))
if currentdir not in sys.path:
    sys.path.insert(0, currentdir) 

In [5]:
# env variables
load_dotenv('../.env.local')

True

In [6]:
from backend.ai import *

In [8]:
what_next('a','b','c')



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:

Answer the following questions as best you can, but speaking as a pirate might
speak. You have access to the following tools:

Search: useful for when you need to answer questions about current events

Use the following format:

Question: the input question you must answer Thought: you should always think
about what to do Action: the action to take, should be one of [Search]
Action Input: the input to the action Observation: the result of the action ...
(this Thought/Action/Action Input/Observation can repeat N times) Thought: I now
know the final answer Final Answer: the final answer to the original input
question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of
"Arg"s

Question: How many people live in canada as of 2023? 
{'prompt_tokens': 178, 'completion_tokens': 37, 'total_tokens': 215}

> Finished chain.
Thought: Arrr, I be needin' to find out the c

The population of Canada as of 2023 is 39,566,248. Arg!
--
AiResult: used 525 token, cost $0.001, took 12.0 seconds